In [4]:
import ktrain
from ktrain import text as txt
import pandas as pd

In [5]:
train = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_same_size_as_bal.csv")
test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [6]:
train.head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg
0,"I don't bank here, but I was shopping in the S...",1.0,NEGATIVE,NEGATIVE,NEGATIVE
1,"I don't appreciate crowds and chaos, but what ...",4.0,POSITIVE,POSITIVE,POSITIVE
2,The beginning of the end. We've been fans of t...,4.0,POSITIVE,POSITIVE,POSITIVE
3,-loud \n- party vibes from Fremont street cont...,5.0,POSITIVE,POSITIVE,POSITIVE
4,My new favorite sushi place in the valley!!! O...,5.0,POSITIVE,POSITIVE,POSITIVE


In [7]:
x_train, y_train = train['text'].tolist(), train['pos_neg_3_is_pos'].tolist()

In [8]:
x_test, y_test = test['text'].tolist(), test['pos_neg_3_is_pos'].tolist()

In [9]:
class_names = train['pos_neg_3_is_pos'].unique().tolist()

In [11]:
trn, val, preproc = txt.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names=class_names,
                                          preprocess_mode='bert',
                                          maxlen=256)

task: text classification
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [12]:
import pickle

pickle.dump( trn, open( "trn.p", "wb" ), protocol=4 )
pickle.dump( val, open( "val.p", "wb" ), protocol=4 )
pickle.dump( preproc, open( "preproc.p", "wb" ), protocol=4 )

In [13]:
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [14]:
bucket = "yelp-dataset-pt-9"

In [15]:
upload_file('trn.p', bucket, 'spencer/data/sentiment/en/ktrain/bert/trn.p')

True

In [16]:
upload_file('val.p', bucket, 'spencer/data/sentiment/en/ktrain/bert/val.p')

True

In [17]:
upload_file('preproc.p', bucket, 'spencer/data/sentiment/en/ktrain/bert/preproc.p')

True